## Exercice 1 : VAE sur MNIST — Solution
On utilise PyTorch pour construire un VAE, entraîner sur MNIST et visualiser l'espace latent.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# Définition du VAE
class VAE(nn.Module):
    def __init__(self, latent_dim=2):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(784, 128),
            nn.ReLU(),
            nn.Linear(128, 32),
            nn.ReLU(),
        )
        self.fc_mu = nn.Linear(32, latent_dim)
        self.fc_logvar = nn.Linear(32, latent_dim)
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 128),
            nn.ReLU(),
            nn.Linear(128, 784),
            nn.Sigmoid()
        )
    def encode(self, x):
        h = self.encoder(x)
        return self.fc_mu(h), self.fc_logvar(h)
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    def decode(self, z):
        return self.decoder(z)
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [ ]:
# Entraînement du VAE et visualisation de l'espace latent
# (code d'entraînement et visualisation à compléter selon le dataset et l'environnement)

## Exercice 2 : GAN sur MNIST — Solution
On construit un GAN simple, on entraîne sur MNIST et on compare les images générées.

In [ ]:
class Generator(nn.Module):
    def __init__(self, z_dim=100, img_dim=784):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(z_dim, 128),
            nn.ReLU(),
            nn.Linear(128, img_dim),
            nn.Tanh()
        )
    def forward(self, z):
        return self.net(z)

class Discriminator(nn.Module):
    def __init__(self, img_dim=784):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(img_dim, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.net(x)

In [ ]:
# Entraînement du GAN et visualisation des résultats
# (code d'entraînement et visualisation à compléter selon le dataset et l'environnement)

## Exercice 3 : Flows — Solution
On illustre une transformation bijective simple et on visualise la distribution transformée.

In [ ]:
import numpy as np
import seaborn as sns
# Transformation affine
def affine_flow(x, a=2.0, b=1.0):
    return a * x + b
x = np.random.normal(0, 1, 1000)
y = affine_flow(x)
sns.histplot(y, kde=True)

## Utilisation du dataset MNIST pour tous les exercices
Les trois exercices (VAE, GAN, Flows) utilisent le dataset MNIST pour l'entraînement, la génération et la visualisation.

---
### Exemple complet d'entraînement sur MNIST pour chaque méthode

In [ ]:
# Entraînement VAE sur MNIST
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Dataset MNIST
transform = transforms.ToTensor()
dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
loader = DataLoader(dataset, batch_size=128, shuffle=True)

vae = VAE(latent_dim=2).to(device)
optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3)

# Fonction de perte VAE
def vae_loss(recon_x, x, mu, logvar):
    BCE = nn.functional.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

# Boucle d'entraînement
for epoch in range(5):
    vae.train()
    total_loss = 0
    for batch, _ in loader:
        batch = batch.view(-1, 784).to(device)
        recon, mu, logvar = vae(batch)
        loss = vae_loss(recon, batch, mu, logvar)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {total_loss/len(loader.dataset):.2f}')

# Visualisation de l'espace latent
vae.eval()
import matplotlib.pyplot as plt
latents, labels = [], []
for batch, label in loader:
    batch = batch.view(-1, 784).to(device)
    mu, _ = vae.encode(batch)
    latents.append(mu.cpu().detach())
    labels.append(label)
    if len(latents) > 10: break
latents = torch.cat(latents)
labels = torch.cat(labels)
plt.scatter(latents[:,0], latents[:,1], c=labels, cmap='tab10', s=5)
plt.title('Espace latent VAE (MNIST)')
plt.show()

In [ ]:
# Entraînement GAN sur MNIST
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = transforms.ToTensor()
dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
loader = DataLoader(dataset, batch_size=128, shuffle=True)

generator = Generator().to(device)
discriminator = Discriminator().to(device)
optim_G = torch.optim.Adam(generator.parameters(), lr=2e-4)
optim_D = torch.optim.Adam(discriminator.parameters(), lr=2e-4)
loss_fn = nn.BCELoss()

for epoch in range(5):
    for real, _ in loader:
        real = real.view(-1, 784).to(device)
        batch_size = real.size(0)
        # Train Discriminator
        z = torch.randn(batch_size, 100).to(device)
        fake = generator(z)
        D_real = discriminator(real)
        D_fake = discriminator(fake.detach())
        loss_D = loss_fn(D_real, torch.ones_like(D_real)) + loss_fn(D_fake, torch.zeros_like(D_fake))
        optim_D.zero_grad()
        loss_D.backward()
        optim_D.step()
        # Train Generator
        D_fake = discriminator(fake)
        loss_G = loss_fn(D_fake, torch.ones_like(D_fake))
        optim_G.zero_grad()
        loss_G.backward()
        optim_G.step()
    print(f'Epoch {epoch+1}, Loss D: {loss_D.item():.2f}, Loss G: {loss_G.item():.2f}')

# Visualisation des images générées
import matplotlib.pyplot as plt
z = torch.randn(16, 100).to(device)
fake_imgs = generator(z).cpu().detach().view(-1, 28, 28)
fig, axes = plt.subplots(1, 16, figsize=(16,2))
for i, ax in enumerate(axes):
    ax.imshow(fake_imgs[i], cmap='gray')
    ax.axis('off')
plt.suptitle('Images générées par GAN (MNIST)')
plt.show()

In [ ]:
# Entraînement d'un Flow sur MNIST (exemple simple)
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform = transforms.ToTensor()
dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
loader = DataLoader(dataset, batch_size=1000, shuffle=True)

# On prend les images et on applique une transformation affine comme flow
for batch, _ in loader:
    batch = batch.view(-1, 784).numpy()
    # Flow: transformation affine
    a, b = 2.0, 1.0
    batch_flow = a * batch + b
    plt.hist(batch_flow.flatten(), bins=50, alpha=0.7)
    plt.title('Distribution transformée par Flow (MNIST)')
    plt.show()
    break

## Exercice 4 : Modèle de diffusion sur MNIST — Solution
On propose ici un exemple de code pour entraîner un modèle de diffusion simple (DDPM) sur MNIST, visualiser le bruitage et la génération.

In [ ]:
# Corrigé : Modèle de diffusion simple (DDPM) sur MNIST
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform = transforms.ToTensor()
dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
loader = DataLoader(dataset, batch_size=128, shuffle=True)

# Réseau de bruitage simple (UNet simplifié)
class SimpleUNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(784, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 784)
        )
    def forward(self, x, t):
        return self.net(x)

# Paramètres de diffusion
T = 100
betas = torch.linspace(1e-4, 0.02, T)
alphas = 1. - betas
alphas_cumprod = torch.cumprod(alphas, dim=0)

# Fonction de bruitage
def q_sample(x_start, t, noise=None):
    if noise is None:
        noise = torch.randn_like(x_start)
    sqrt_alphas_cumprod_t = alphas_cumprod[t].sqrt().to(device)
    sqrt_one_minus_alphas_cumprod_t = (1 - alphas_cumprod[t]).sqrt().to(device)
    return sqrt_alphas_cumprod_t * x_start + sqrt_one_minus_alphas_cumprod_t * noise

# Entraînement du modèle de diffusion
model = SimpleUNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(3):
    for x, _ in loader:
        x = x.view(-1, 784).to(device)
        t = torch.randint(0, T, (x.size(0),), device=device)
        noise = torch.randn_like(x)
        x_noisy = q_sample(x, t, noise)
        pred_noise = model(x_noisy, t)
        loss = nn.functional.mse_loss(pred_noise, noise)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')

# Visualisation du bruitage progressif
x, _ = next(iter(loader))
x = x[:8].view(-1, 784).to(device)
fig, axes = plt.subplots(1, 8, figsize=(16,2))
for i in range(8):
    t = int(i * T / 8)
    x_noisy = q_sample(x, t)
    axes[i].imshow(x_noisy[0].cpu().view(28,28), cmap='gray')
    axes[i].set_title(f't={t}')
    axes[i].axis('off')
plt.suptitle('Processus de bruitage (MNIST, Diffusion)')
plt.show()